In [1]:
import os
from pathlib import Path


from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings




In [2]:
# from pathlib import Path

# project_path: Path = Path("./demo_app")

# import glob

# # List to store relative paths of all files in the demo_app folder
# file_paths = []

# # Glob through all files in the demo_app folder
# for filepath in glob.glob("demo_app/**", recursive=True):
#     if os.path.isfile(filepath):  # Ensure it's a file and not a directory
#         # Check the file extension and skip if it's an image format
#         ext = os.path.splitext(filepath)[1].lower()
#         if ext not in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']:
#             file_paths.append(filepath)

# # Print the list of file paths
# print(file_paths)


In [3]:
loader = DirectoryLoader("./demo_app", glob="**/*.*", show_progress=True, use_multithreading=True, loader_cls=TextLoader)
docs = loader.load()


 93%|█████████▎| 151/163 [00:00<00:00, 9869.87it/s]


In [4]:
len(docs)


137

In [5]:
model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [6]:
from langchain.document_loaders import TextLoader


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

embeddings = hf
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

Created a chunk of size 1187, which is longer than the specified 1000
Created a chunk of size 4252, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 3525, which is longer than the specified 1000
Created a chunk of size 1388, which is longer than the specified 1000
Created a chunk of size 1559, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 4272, which is longer than the specified 1000
Created a chunk of size 2128, which is longer than the specified 1000
Created a chunk of size 3427, which is longer than the specified 1000
Created a chunk of size 1091, which is longer than the specified 1000
Created a chunk of size 1161, which is longer than the specified 1000
Created a chunk of size 1778, which is longer than the specified 1000
Created a chunk of size 1998, which is longer than the specified 1000
Created a chunk of s

In [10]:
query = "dockerfile for backend"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

{'source': 'demo_app/docker-compose.override.yml'}


In [ ]:
code_base = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)